In [63]:
import pandas as pd
import mysql.connector as db
import json
from base64 import b64encode
import os

import ipywidgets as widgets
from IPython.display import display, HTML

with open('config.json') as f:
    config = json.load(f)
    
mydb = db.connect(host=config['host'], user=config['user'], passwd=config['passwd'], port=9453)
cursor = mydb.cursor()


cursor.execute("USE newImageDB;")

In [64]:
# For Visualization
def SQL_COMMAND(cursor, query):
    
    cursor.execute(query)
    rows = cursor.fetchall()
    columns = [i[0] for i in cursor.description]

    df = pd.DataFrame(rows, columns=columns)

    html_str = '<table border="1" style="solid; border-collapse: collapse; width: 100%; table-layout: fixed;">'
    html_str += '<tr>' + ''.join(f'<th style="padding: 8px; text-align: center; background-color: #323232;">{col}</th>' for col in df.columns) + '</tr>'
    for index, row in df.iterrows():
        html_str += '<tr>'
        for col in df.columns:
            if col == 'img_path':
                # /Users/chian/Documents/DBMS/DBMS_Final
                with open(row[col].replace('/mysqludf/', '/Users/chian/Documents/DBMS/DBMS_Final/過程廢物/'), "rb") as image_file:
                    encoded_string = b64encode(image_file.read()).decode()
                img_html = f'<img src="data:image/jpeg;base64,{encoded_string}" style="height: 100px; display: block; margin: auto;"/>'
                html_str += f'<td style="padding: 8px;">{img_html}</td>'
            else:
                html_str += f'<td style="padding: 8px; text-align: center;">{row[col]}</td>'
        html_str += '</tr>'
    html_str += '</table>'

    display(HTML(html_str))

## Insert Test Data to DB
10 classes x 5 images = 50

In [65]:
info = {
"n02690373": {
    "class_name" : "airliner",
    "number": 0
},
"n04285008":{
    "class_name" : "sports_car",
    "number": 1
},
"n01833805":{
    "class_name" : "hummingbird", 
    "number": 2
},
"n02124075":{
    "class_name" : "Egyptian_cat", 
    "number": 3
},
"n02403003":{
    "class_name" : "Ox",
    "number": 4
},
"n02099601":{
    "class_name" : "golden_retriever",
    "number": 5
},
"n01644900":{
    "class_name" : "tailed_frog",
    "number": 6
},
"n02391049":{
    "class_name" : "zebra",
    "number": 7
},
"n03095699":{
    "class_name" : "container_ship",
    "number": 8
},
"n04467665":{
    "class_name" : "trailer_truck",
    "number": 9
}
}

In [67]:
insert_stmt = (
        "INSERT INTO image_table (img_path, ground_truth) VALUES (%s, %s)"
    )

root_dir = 'imgs'
categories = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]

selected_images = []

for category in categories:
    category_path = os.path.join(root_dir, category)
    images = [img for img in os.listdir(category_path) if img.endswith('.JPEG')]

    images = sorted(images)
    selected_images = images[:5]
    
    ground_truth = info[category]['number']

    for image in selected_images:
        cursor.execute(insert_stmt, (os.path.join('/mysqludf', root_dir, category, image), ground_truth))

In [69]:
def my_function(b):
    try:
        param = str(text_input.value)
        SQL_COMMAND(cursor=cursor, query=param)
    except ValueError:
        print("SQL COMMAND INVALID!")
        
text_input = widgets.Text(
    value='SELECT * FROM image_table;',
    description='SQL',
    disabled=False,
    layout=widgets.Layout(width='100%') 
)

button = widgets.Button(
    description='Submit',
    button_style='success', 
    icon='check',
    layout=widgets.Layout(width='100px', height='30px')
)

button.on_click(my_function)
display(widgets.HBox([text_input, button]))

## Spend & Accuracy Test

about 1 hour for 4 setting, [oringinal, fast, faster, fastest]

In [70]:
def calculate_acc(rows, GD):
    rows = rows[:10]
    
    acc = 0
    for row in rows:
        # print(row)
        if row[0] == GD:
            acc += 1
    return acc * 20

In [76]:
import time

root_dir = 'imgs'
categories = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]

selected_images = []

for category in categories:
    category_path = os.path.join(root_dir, category)
    images = [img for img in os.listdir(category_path) if img.endswith('.JPEG')]
    images = sorted(images)

    selected_images = [images[11]]
    ground_truth = info[category]['number']

    for image in selected_images:

        print(f'Class: {ground_truth}')
        img_path = os.path.join('/mysqludf', root_dir, category, image)

        time0 = time.time()
        cursor.execute(f"SELECT ground_truth, img_path, image_sim(img_path, '{img_path}') AS sim FROM image_table;")
        rows = cursor.fetchall()
        Acc0 = calculate_acc(rows, ground_truth)

        time1 = time.time()
        cursor.callproc("fast_sim", (img_path, )) 
        rows = []
        for result in cursor.stored_results():
            rows.extend(result.fetchall())
        Acc1 = calculate_acc(rows, ground_truth)
        
        time2 = time.time()
        cursor.callproc("faster_sim", (img_path, )) 
        rows = []
        for result in cursor.stored_results():
            rows.extend(result.fetchall())
        Acc2 = calculate_acc(rows, ground_truth)

        time3 = time.time()
        cursor.callproc("fastest_sim", (img_path, )) 
        time4 = time.time()
        rows = []
        for result in cursor.stored_results():
            rows.extend(result.fetchall())
        Acc3 = calculate_acc(rows, ground_truth)

        print(f'===============================')
        print(f'Original: {time1 - time0} sec, Original Acc: {Acc0}')
        print(f'Fast: {time2 - time1} sec, Fast Acc: {Acc1}')
        print(f'Faster: {time3 - time2} sec, Faster Acc: {Acc2}')
        print(f'Fastest: {time4 - time3} sec, Fastest Acc: {Acc3}')
    break

Class: 3
Original: 114.07613778114319 sec, Original Acc: 100
Fast: 78.04226303100586 sec, Fastest Acc: 100
Faster: 67.18326306343079 sec, Fastest Acc: 100
Fastest: 37.662351846694946 sec, Fastest Acc: 100


In [44]:
# cursor.execute("DELETE FROM image_table;")
cursor.execute("TRUNCATE TABLE image_table;")

In [6]:
mydb.commit()
cursor.close()
mydb.close()